### Installing Required Libraries

In [1]:
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-2.11                  |         openblas          10 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    scipy-1.3.2                |   py36h921218d_0        18.0 MB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-for

In [2]:
!conda install -c conda-forge lxml --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.4.1                 |   py36h7ec2d77_0         1.6 MB  conda-forge

The following packages will be UPDATED:

    lxml: 4.2.5-py37hefd8a0e_0 --> 4.4.1-py36h7ec2d77_0 conda-forge


lxml-4.4.1           | 1.6 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [3]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.50   | 

### Importing Required Libraries

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# to access wikipedia pages and perform required clenaup
from bs4 import BeautifulSoup
import lxml

import requests # library to handle requests

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

###!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
###!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


###   

## Part # 1

### I) Accessing Wikipedia page to download Canada zip code data, perform required cleanup, transformation, and load data into pandas dataframe

In [5]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'html.parser')
#print(soup.prettify())

### II) Locate zipcode table from wikipedia page, review HTML content, and parse data to capture content and load into pandas dataframe

In [6]:
#<table class="wikitable sortable">
cnd_zip_table = soup.find('table',{'class':'wikitable sortable'})
#cnd_zip_table

In [7]:
# identifying number of lines
row_marker = 0
for row in cnd_zip_table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    if len(columns) > 0:
        row_marker += 1    

# creating dataframe to capture table details from web url
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

table_df = pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'], index=range(0,row_marker))
#table_df.shape

# parsing data and loading into dataframe
row_marker = 0
for row in cnd_zip_table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        myText=column.get_text()
        table_df.iat[row_marker,column_marker] = myText.strip('\t\r\n') # removing new line character
        column_marker = column_marker+ 1
    if len(columns) > 0:
        row_marker += 1

print('table_df shape :', table_df.shape)
table_df.head()

table_df shape : (287, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### III) Perform required cleanup to obtain desired output
#### 1) Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
#### 2) More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
#### 3) If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [8]:
# step 1
temp_df = table_df[table_df['Borough']!='Not assigned']
print('temp_df shape :', temp_df.shape)

# step 2
temp_grouped_df = temp_df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join).reset_index()
print('temp_grouped_df shape : ', temp_grouped_df.shape)
temp_grouped_df.head()

temp_df shape : (210, 3)
temp_grouped_df shape :  (103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
# identify which postal code neighbourhood is having Not assigned value
temp_grouped_df[temp_grouped_df['Neighborhood']=='Not assigned']

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Not assigned


In [10]:
# step 3
try:
    temp_grouped_df.at[temp_grouped_df[temp_grouped_df['Neighborhood']=='Not assigned'].index[0],'Neighborhood']=temp_grouped_df.at[temp_grouped_df[temp_grouped_df['Neighborhood']=='Not assigned'].index[0],'Borough']
except:
    pass  # no change to data
#temp_grouped_df.ix[85]
'''
PostalCode                M7A
Borough          Queen's Park
Neighbourhood    Queen's Park
Name: 85, dtype: object
'''
temp_grouped_df.head(5)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
print('temp_grouped_df shape : ', temp_grouped_df.shape)

temp_grouped_df shape :  (103, 3)


###    

###   

## Part # 2

### Built a dataframe of the postal code of each neighborhood along with the borough name, neighborhood name, latitude and the longitude coordinates of each neighborhood.

### I) Download lat/ long data from csv file, create dataframe using pandas read_csv utility, and join it with earlier created neighbourhood dataframe

In [15]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
print('Data downloaded!')

canada_latlng_df = pd.read_csv('Geospatial_Coordinates.csv', names=['PostalCode','Latitude','Longitude'])

canada_latlng_df = canada_latlng_df[1:]

Data downloaded!


In [16]:
canada_latlng_df.head()

,PostalCode,Latitude,Longitude
1,M1B,43.8066863,-79.1943534
2,M1C,43.7845351,-79.1604971
3,M1E,43.7635726,-79.1887115
4,M1G,43.7709921,-79.2169174
5,M1H,43.773136,-79.2394761


In [19]:
# join both dataframes to produce complete record
canada_df = temp_grouped_df.set_index('PostalCode').join(canada_latlng_df.set_index('PostalCode'), on='PostalCode',how='left').reset_index()

In [20]:
canada_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.8066863,-79.1943534
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.7845351,-79.1604971
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.7635726,-79.1887115
3,M1G,Scarborough,Woburn,43.7709921,-79.2169174
4,M1H,Scarborough,Cedarbrae,43.773136,-79.2394761


In [21]:
canada_df.shape

(103, 5)

###   

### Part 3

### I) Explore and cluster the neighborhoods in Toronto.

In [22]:
canada_toronto_df = canada_df[canada_df['Borough'].str.contains('Toronto')]
canada_toronto_df.shape

(38, 5)

In [23]:
canada_toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.6763574,-79.2930312
41,M4K,East Toronto,"The Danforth West,Riverdale",43.6795571,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.6689985,-79.3155716
43,M4M,East Toronto,Studio District,43.6595255,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.7280205,-79.3887901


In [24]:
CLIENT_ID = '1MARTJCYWAQJAQO14JYSGURNA0FF1NCNJIBNXZK3O5TJZFMT' # your Foursquare ID
CLIENT_SECRET = '3KPE1BKCVEQ2H2TTXOGPFALUONUSES31YUGEC4HJRO2CERXQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

LIMIT = 100
radius=500

Your credentails:
CLIENT_ID: 1MARTJCYWAQJAQO14JYSGURNA0FF1NCNJIBNXZK3O5TJZFMT
CLIENT_SECRET:3KPE1BKCVEQ2H2TTXOGPFALUONUSES31YUGEC4HJRO2CERXQ


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
     
        '''
        for venue_list in venues_list:
            for item in venue_list:
                print (item)
        '''     
        # generating list from venues
        #print([item for venue_list in venues_list for item in venue_list])

    # generating list from venues and creating DataFrame from list
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    #nearby_venues

    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### II) Use Foursquare API to pull required dataset

In [26]:
toronto_venues = getNearbyVenues(names=canada_toronto_df['Neighborhood'],
                                   latitudes=canada_toronto_df['Latitude'],
                                   longitudes=canada_toronto_df['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede

In [37]:
print('toronto_venues Shape:',toronto_venues.shape)
toronto_venues.head()

toronto_venues Shape: (1686, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.6763574,-79.2930312,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.6763574,-79.2930312,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.6763574,-79.2930312,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.6763574,-79.2930312,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.6795571,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [38]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.


### III) Perform One Hot Encoding ; needed for clustering algo to work e.g. numerical data set. Group data by Neighborhood

In [39]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print('toronto_onehot shape : ', toronto_onehot.shape)
toronto_onehot.head()

toronto_onehot shape :  (1686, 233)


,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [40]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

#toronto_grouped = toronto_onehot.groupby('Neighborhood').sum().reset_index()
# adding total number of venues
#toronto_grouped_total = toronto_grouped.sum(axis=1)
#toronto_grouped.insert(1,'Total_Venues',toronto_grouped_total)


,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,King,Richmond",0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.03,0.0,0.0,0.0,0.010000,0.0,0.03,0.0,0.0000,0.000000,0.0,0.000000,0.030000,0.0,0.030000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.01,0.0000,0.01,0.020000,0.0000,0.0,0.01,0.03,0.010000,0.0,0.000000,0.050000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.070000,0.0,0.0,0.0,0.01,0.000000,0.0000,0.020000,0.000000,0.03,0.0,0.000000,0.0,0.0,0.0,0.01,0.01,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.01,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.01,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0000,0.0000,0.02,0.0,

In [41]:
toronto_grouped.shape

(38, 233)

### IV) Concat Neightborhood with number of venue to visulize impact of number of venue on cluster

In [42]:
# selecting column name based on sorted values
# toronto_grouped.iloc[0,:10].index.values[0:5]
#array(['Neighborhood', 'Accessories Store', 'Adult Boutique','Afghan Restaurant', 'African Restaurant'], dtype=object)

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    x1=[]
    for i,v in zip(row_categories_sorted.index.values[0:num_top_venues],row_categories_sorted.values[0:num_top_venues]):
        x1.append(str(i) + '(' + str(round(v,4)) + ')')
    return x1
    #return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
#columns = ['Neighborhood','Total_Venues']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
#neighborhoods_venues_sorted[['Neighborhood','Total_Venues']] = toronto_grouped[['Neighborhood','Total_Venues']]
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    #neighborhoods_venues_sorted.iloc[ind, 2:] = return_most_common_venues(toronto_grouped.iloc[ind, 1:], num_top_venues)
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, 1:], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop(0.07),Café(0.05),Steakhouse(0.04),Thai Restaurant(0.04),Asian Restaurant(0.03),Cosmetics Shop(0.03),Restaurant(0.03),Burger Joint(0.03),American Restaurant(0.03),Bar(0.03)
1,Berczy Park,Coffee Shop(0.0702),Bakery(0.0526),Seafood Restaurant(0.0351),Cheese Shop(0.0351),Beer Bar(0.0351),Cocktail Bar(0.0351),Steakhouse(0.0351),Café(0.0351),Farmers Market(0.0351),Breakfast Spot(0.0175)
2,"Brockton,Exhibition Place,Parkdale Village",Breakfast Spot(0.0952),Coffee Shop(0.0952),Café(0.0952),Performing Arts Venue(0.0476),Climbing Gym(0.0476),Stadium(0.0476),Sandwich Place(0.0476),Burrito Place(0.0476),Restaurant(0.0476),Pet Store(0.0476)
3,Business Reply Mail Processing Centre 969 Eastern,Smoke Shop(0.0625),Pizza Place(0.0625),Burrito Place(0.0625),Auto Workshop(0.0625),Spa(0.0625),Fast Food Restaurant(0.0625),Farmers Market(0.0625),Garden(0.0625),Garden Center(0.0625),Brewery(0.0625)
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service(0.125),Airport Terminal(0.125),Coffee Shop(0.0625),Bar(0.0625),Plane(0.0625),Rental Car Location(0.0625),Sculpture Garden(0.0625),Boutique(0.0625),Boat or Ferry(0.0625),Harbor / Marina(0.0625)


### V) Use KMean Cluster ML algo to Cluster Neighborhoods

In [44]:
# set number of clusters; Ten
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 1, 7, 3, 3, 3, 1, 3], dtype=int32)

### VI) Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [45]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = canada_toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(10) # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.6763574,-79.2930312,8,Health Food Store(0.25),Trail(0.25),Pub(0.25),Wings Joint(0.0),Dessert Shop(0.0),Event Space(0.0),Ethiopian Restaurant(0.0),Electronics Store(0.0),Eastern European Restaurant(0.0),Dumpling Restaurant(0.0)
41,M4K,East Toronto,"The Danforth West,Riverdale",43.6795571,-79.352188,3,Greek Restaurant(0.2093),Coffee Shop(0.093),Italian Restaurant(0.0698),Ice Cream Shop(0.0698),Restaurant(0.0465),Bookstore(0.0465),Furniture / Home Store(0.0465),Bakery(0.0233),Pizza Place(0.0233),Brewery(0.0233)
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.6689985,-79.3155716,1,Park(0.1),Board Shop(0.05),Ice Cream Shop(0.05),Brewery(0.05),Sandwich Place(0.05),Burger Joint(0.05),Burrito Place(0.05),Fast Food Restaurant(0.05),Fish & Chips Shop(0.05),Italian Restaurant(0.05)
43,M4M,East Toronto,Studio District,43.6595255,-79.340923,3,Café(0.1081),Coffee Shop(0.0811),Bakery(0.0541),Italian Restaurant(0.0541),American Restaurant(0.0541),Clothing Store(0.027),Diner(0.027),Cheese Shop(0.027),Bar(0.027),Bank(0.027)
44,M4N,Central Toronto,Lawrence Park,43.7280205,-79.3887901,4,Bus Line(0.3333),Park(0.3333),Swim School(0.3333),Wings Joint(0.0),Event Space(0.0),Ethiopian Restaurant(0.0),Electronics Store(0.0),Eastern European Restaurant(0.0),Dumpling Restaurant(0.0),Donut Shop(0.0)
45,M4P,Central Toronto,Davisville North,43.7127511,-79.3901975,1,Breakfast Spot(0.1111),Park(0.1111),Gym(0.1111),Pizza Place(0.1111),Food & Drink Shop(0.1111),Clothing Store(0.1111),Convenience Store(0.1111),Sandwich Place(0.1111),Hotel(0.1111),Doner Restaurant(0.0)
46,M4R,Central Toronto,North Toronto West,43.7153834,-79.4056784,3,Coffee Shop(0.1053),Clothing Store(0.1053),Sporting Goods Shop(0.1053),Miscellaneous Shop(0.0526),Restaurant(0.0526),Mexican Restaurant(0.0526),Burger Joint(0.0526),Dessert Shop(0.0526),Salon / Barbershop(0.0526),Chinese Restaurant(0.0526)
47,M4S,Central Toronto,Davisville,43.7043244,-79.3887901,1,Sandwich Place(0.0811),Pizza Place(0.0811),Dessert Shop(0.0811),Coffee Shop(0.0541),Gym(0.0541),Italian Restaurant(0.0541),Sushi Restaurant(0.0541),Café(0.0541),Indian Restaurant(0.027),New American Restaurant(0.027)
48,M4T,Central Toronto,"Moore Park,Summerhill East",43.6895743,-79.3831599,5,Gym(0.5),Playground(0.5),Wings Joint(0.0),Dessert Shop(0.0),Event Space(0.0),Ethiopian Restaurant(0.0),Electronics Store(0.0),Eastern European Restaurant(0.0),Dumpling Restaurant(0.0),Donut Shop(0.0)
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.6864123,-79.4000493,3,Coffee Shop(0.1333),Pub(0.1333),Sushi Restaurant(0.0667),Pizza Place(0.0667),Liquor Store(0.0667),Restaurant(0.0667),Sports Bar(0.0667),Bagel Shop(0.0667),Supermarket(0.0667),Fried Chicken Joint(0.0667)


In [46]:
toronto_merged.shape

(38, 16)

In [47]:
canada_toronto_df.shape

(38, 5)

## VII) Visualize clusters using Folium

In [48]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [49]:
# changing data type of Lat/Long to float ; required to draw map
toronto_merged['Latitude'] = toronto_merged['Latitude'].astype('float')
toronto_merged['Longitude'] = toronto_merged['Longitude'].astype('float')
#toronto_merged.dtypes

In [50]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [53]:
toronto_merged['Cluster Labels'].value_counts()

3    21
1     9
9     1
8     1
7     1
6     1
5     1
4     1
2     1
0     1
Name: Cluster Labels, dtype: int64

### Except 3 & 1, all the clusters are having only one candidate. If cluster are reduced to 5, distribution of members to cluster have similar behavior. As we reduce number of clusters, most of the members are assigned to one or two clusters. Based on available data for Neighbor, it is difficult to identify meaningful cluster using unsupervised algorithm.

In [55]:
# list all members of cluster 1
toronto_merged[toronto_merged['Cluster Labels']==1]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,1,Park(0.1),Board Shop(0.05),Ice Cream Shop(0.05),Brewery(0.05),Sandwich Place(0.05),Burger Joint(0.05),Burrito Place(0.05),Fast Food Restaurant(0.05),Fish & Chips Shop(0.05),Italian Restaurant(0.05)
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Breakfast Spot(0.1111),Park(0.1111),Gym(0.1111),Pizza Place(0.1111),Food & Drink Shop(0.1111),Clothing Store(0.1111),Convenience Store(0.1111),Sandwich Place(0.1111),Hotel(0.1111),Doner Restaurant(0.0)
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,1,Sandwich Place(0.0811),Pizza Place(0.0811),Dessert Shop(0.0811),Coffee Shop(0.0541),Gym(0.0541),Italian Restaurant(0.0541),Sushi Restaurant(0.0541),Café(0.0541),Indian Restaurant(0.027),New American Restaurant(0.027)
65,M5R,Central Toronto,"The Annex,North Midtown,Yorkville",43.672710,-79.405678,1,Sandwich Place(0.15),Café(0.15),Coffee Shop(0.1),History Museum(0.05),Burger Joint(0.05),Pub(0.05),Indian Restaurant(0.05),BBQ Joint(0.05),Pizza Place(0.05),Pharmacy(0.05)
66,M5S,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049,1,Café(0.1429),Sandwich Place(0.0571),Restaurant(0.0571),Bookstore(0.0571),Japanese Restaurant(0.0571),Italian Restaurant(0.0571),Bar(0.0571),Bakery(0.0571),French Restaurant(0.0286),Pub(0.0286)
75,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1,Grocery Store(0.1765),Café(0.1765),Park(0.1176),Baby Store(0.0588),Nightclub(0.0588),Restaurant(0.0588),Athletics & Sports(0.0588),Diner(0.0588),Italian Restaurant(0.0588),Candy Store(0.0588)
82,M6P,West Toronto,"High Park,The Junction South",43.661608,-79.464763,1,Bar(0.08),Mexican Restaurant(0.08),Grocery Store(0.08),Café(0.08),Thai Restaurant(0.08),Diner(0.04),Bakery(0.04),Cajun / Creole Restaurant(0.04),Speakeasy(0.04),Fried Chicken Joint(0.04)
84,M6S,West Toronto,"Runnymede,Swansea",43.651571,-79.484450,1,Café(0.0833),Pizza Place(0.0833),Coffee Shop(0.0833),Italian Restaurant(0.0556),Sushi Restaurant(0.0556),Restaurant(0.0278),Smoothie Shop(0.0278),Diner(0.0278),Food(0.0278),Bar(0.0278)
87,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,1,Smoke Shop(0.0625),Pizza Place(0.0625),Burrito Place(0.0625),Auto Workshop(0.0625),Spa(0.0625),Fast Food Restaurant(0.0625),Farmers Market(0.0625),Garden(0.0625),Garden Center(0.0625),Brewery(0.0625)


In [56]:
# list all members of cluster 3
toronto_merged[toronto_merged['Cluster Labels']==3] #.sort_values(by=['Total_Venues'], ascending=False)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,3,Greek Restaurant(0.2093),Coffee Shop(0.093),Italian Restaurant(0.0698),Ice Cream Shop(0.0698),Restaurant(0.0465),Bookstore(0.0465),Furniture / Home Store(0.0465),Bakery(0.0233),Pizza Place(0.0233),Brewery(0.0233)
43,M4M,East Toronto,Studio District,43.659526,-79.340923,3,Café(0.1081),Coffee Shop(0.0811),Bakery(0.0541),Italian Restaurant(0.0541),American Restaurant(0.0541),Clothing Store(0.027),Diner(0.027),Cheese Shop(0.027),Bar(0.027),Bank(0.027)
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,3,Coffee Shop(0.1053),Clothing Store(0.1053),Sporting Goods Shop(0.1053),Miscellaneous Shop(0.0526),Restaurant(0.0526),Mexican Restaurant(0.0526),Burger Joint(0.0526),Dessert Shop(0.0526),Salon / Barbershop(0.0526),Chinese Restaurant(0.0526)
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,3,Coffee Shop(0.1333),Pub(0.1333),Sushi Restaurant(0.0667),Pizza Place(0.0667),Liquor Store(0.0667),Restaurant(0.0667),Sports Bar(0.0667),Bagel Shop(0.0667),Supermarket(0.0667),Fried Chicken Joint(0.0667)
51,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,3,Coffee Shop(0.0652),Restaurant(0.0435),Bakery(0.0435),Pub(0.0435),Café(0.0435),Park(0.0435),Pizza Place(0.0435),Market(0.0435),Italian Restaurant(0.0435),Chinese Restaurant(0.0435)
52,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,3,Coffee Shop(0.0843),Japanese Restaurant(0.0602),Sushi Restaurant(0.0602),Gay Bar(0.0482),Restaurant(0.0361),Men's Store(0.0241),Gastropub(0.0241),Gym(0.0241),Hotel(0.0241),Mediterranean Restaurant(0.0241)
53,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,3,Coffee Shop(0.1429),Park(0.0612),Pub(0.0612),Bakery(0.0612),Mexican Restaurant(0.0408),Café(0.0408),Restaurant(0.0408),Theater(0.0408),Breakfast Spot(0.0408),Farmers Market(0.0204)
54,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,3,Coffee Shop(0.08),Clothing Store(0.07),Cosmetics Shop(0.03),Fast Food Restaurant(0.03),Café(0.03),Bakery(0.03),Middle Eastern Restaurant(0.03),Tea Room(0.02),Hotel(0.02),Pizza Place(0.02)
55,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Café(0.06),Restaurant(0.05),Coffee Shop(0.05),Clothing Store(0.04),Italian Restaurant(0.04),Hotel(0.04),Bakery(0.03),Diner(0.03),Beer Bar(0.03),Cosmetics Shop(0.03)
56,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,3,Coffee Shop(0.0702),Bakery(0.0526),Seafood Restaurant(0.0351),Cheese Shop(0.0351),Beer Bar(0.0351),Cocktail Bar(0.0351),Steakhouse(0.0351),Café(0.0351),Farmers Market(0.0351),Breakfast Spot(0.0175)


In [58]:
# list all members of cluster 0
toronto_merged[toronto_merged['Cluster Labels']==0] #.sort_values(by=['Total_Venues'], ascending=False)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park(0.5),Playground(0.25),Trail(0.25),Department Store(0.0),Event Space(0.0),Ethiopian Restaurant(0.0),Electronics Store(0.0),Eastern European Restaurant(0.0),Dumpling Restaurant(0.0),Donut Shop(0.0)


In [59]:
# list all members of cluster 1
toronto_merged[toronto_merged['Cluster Labels']==2]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,M5N,Central Toronto,Roselawn,43.711695,-79.416936,2,Garden(1.0),Dessert Shop(0.0),Falafel Restaurant(0.0),Event Space(0.0),Ethiopian Restaurant(0.0),Electronics Store(0.0),Eastern European Restaurant(0.0),Dumpling Restaurant(0.0),Donut Shop(0.0),Doner Restaurant(0.0)


In [60]:
# list all members of cluster 4
toronto_merged[toronto_merged['Cluster Labels']==4]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,M4N,Central Toronto,Lawrence Park,43.72802,-79.38879,4,Bus Line(0.3333),Park(0.3333),Swim School(0.3333),Wings Joint(0.0),Event Space(0.0),Ethiopian Restaurant(0.0),Electronics Store(0.0),Eastern European Restaurant(0.0),Dumpling Restaurant(0.0),Donut Shop(0.0)


### Thought of performing clustering using KNN, however, it is supervised learning, so we need labels / target. Due to lack of subject matter expert and meaningful dataset, I am unable to labels data correctly. Alternatively, I can use KMean result and feed it to KNN ML algorithm. However, it will not produce better result than KMean. Since, I am using KMean data to teach KNN. So, it is chicken or egg situation 😊